In [1]:
from pandas import Series, DataFrame
import numpy as np
import pandas as pd

In [2]:
a= Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan], index=list('fedcba'))
a

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

In [4]:
b= Series(np.arange(len(a), dtype= np.float64), index= list('fedcba'))
b[-1]= np.nan
b

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    NaN
dtype: float64

In [6]:
np.where(pd.isnull(a),b,a)

array([ 0. ,  2.5,  2. ,  3.5,  4.5,  nan])

In [7]:
b[:-2]

f    0.0
e    1.0
d    2.0
c    3.0
dtype: float64

In [8]:
a[2:]

d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

In [11]:
b[:-2].combine_first(a[2:])

a    NaN
b    4.5
c    3.0
d    2.0
e    1.0
f    0.0
dtype: float64

In [12]:
df1= DataFrame({'a': [1, np.nan, 5, np.nan],
                'b': [np.nan, 2, np.nan, 6],
                'c': range(2,18,4)})
df1

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [13]:
df2= DataFrame({'a': [5, 4, np.nan, 3, 7],
                'b': [np.nan, 2, 4, 6, 8]})
df2

,a,b
0,5.0,NaN
1,4.0,2.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [15]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


Reshaping and Pivoting
---

In [2]:
data = DataFrame(np.arange(6).reshape((2,3)),
                 index = pd.Index(['Ohio','Colorado'], name='states'),
                 columns = pd.Index(['one','two','three'], name='number'))
data

number,one,two,three
states,,,
Ohio,0,1,2
Colorado,3,4,5


In [3]:
result = data.stack()
result

states    number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

In [4]:
result.unstack()

number,one,two,three
states,,,
Ohio,0,1,2
Colorado,3,4,5


In [5]:
result.unstack('states')

states,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [6]:
s1 = Series([0,1,2,3], index=list('abcd'))
s2= Series([4,5,6], index=['c','d','e'])
data2= pd.concat([s1,s2], keys=['one','two'])
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [7]:
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [8]:
data2.unstack().stack()

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [9]:
# if we don't want to remove NaN
data2.unstack().stack(dropna=False)

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

In [28]:
data

number,one,two,three
states,,,
Ohio,0,1,2
Colorado,3,4,5


In [11]:
df = DataFrame({'left': result, 'right': result+5}, columns = pd.Index(['left','right'], name = 'side'))
df

side             left  right
states   number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [14]:
df.unstack('states')

side   left          right         
states Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [15]:
df.unstack('states').stack('side')

states        Ohio  Colorado
number side                 
one    left      0         3
       right     5         8
two    left      1         4
       right     6         9
three  left      2         5
       right     7        10

In [16]:
ldata[:10]

NameError: name 'ldata' is not defined

DATA TRANSFORMATION
---

***Removing Duplicates***

In [2]:
data = DataFrame({'k1': ['one']*3 + ['two']*4,
                  'k2': [1,1,2,3,3,4,4]})
data

,k1,k2
0,one,1
1,one,1
2,one,2
3,two,3
4,two,3
5,two,4
6,two,4


In [3]:
data.duplicated()

0    False
1     True
2    False
3    False
4     True
5    False
6     True
dtype: bool

In [5]:
data.drop_duplicates()

,k1,k2
0,one,1
2,one,2
3,two,3
5,two,4


In [6]:
data['v1']= range(7)

In [7]:
data.drop_duplicates('k1')

,k1,k2,v1
0,one,1,0
3,two,3,3


In [9]:
data.drop_duplicates(['k1','k2'], keep='last')

,k1,k2,v1
1,one,1,1
2,one,2,2
4,two,3,4
6,two,4,6


**Transforming data using a function or mapping**

In [10]:
data= DataFrame({'food': ['bacon', 'pulled pork', 'bacon', 'Pastrami', 'corned beef',
                          'Bacon', 'pastrami', 'honey ham', 'nova lox'],
                 'ounces': [4,3,12,6,7.5,8,3,5,6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [11]:
meat_to_animal = {
    'bacon': 'pig',
    'pulled pork': 'pig',
    'pastrami': 'cow',
    'corned beef': 'cow',
    'honey ham': 'pig',
    'nova lox': 'salmon'
}

In [12]:
data['animal'] = data['food'].map(str.lower).map(meat_to_animal)

In [13]:
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [14]:
data = Series([1,-999,2,-999,-1000,3])
data

0       1
1    -999
2       2
3    -999
4   -1000
5       3
dtype: int64

In [16]:
data.replace([-999,-1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [17]:
data.replace([-999,-1000],[np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [18]:
data = DataFrame(np.arange(12).reshape((3,4)),
                 index=['Ohio','Colorado','New York'],
                 columns=['one','two','three','four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [19]:
data.index.map(str.upper)

array(['OHIO', 'COLORADO', 'NEW YORK'], dtype=object)

In [20]:
data.index = data.index.map(str.upper)

In [21]:
data

,one,two,three,four
OHIO,0,1,2,3
COLORADO,4,5,6,7
NEW YORK,8,9,10,11


In [22]:
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [28]:
data.rename(index={'OHIO': 'INDIANA'}, columns= {'three': 'peekaboo'})


,one,two,peekaboo,four
INDIANA,0,1,2,3
COLORADO,4,5,6,7
NEW YORK,8,9,10,11


In [24]:
data

,one,two,three,four
OHIO,0,1,2,3
COLORADO,4,5,6,7
NEW YORK,8,9,10,11
